# Get the data

In [2]:
import os
import pandas as pd
import requests

## Best movies from IMDB

Simple scrape. This is on the edge of legal use of the website.

In [37]:
# Using request because without the proper HTTP header, I am getting Czech titles
url = "https://www.imdb.com/chart/top/"
response = requests.get(url, headers={
    "Accept-Language": "en"
})
raw_df = pd.read_html(response.content)[0]
raw_df

,Unnamed: 0,Rank & Title,IMDb Rating,Your Rating,Unnamed: 4
0,NaN,1. The Shawshank Redemption (1994),9.2,12345678910 NOT YET RELEASED Seen,NaN
1,NaN,2. The Godfather (1972),9.2,12345678910 NOT YET RELEASED Seen,NaN
2,NaN,3. The Dark Knight (2008),9.0,12345678910 NOT YET RELEASED Seen,NaN
3,NaN,4. The Godfather Part II (1974),9.0,12345678910 NOT YET RELEASED Seen,NaN
4,NaN,5. 12 Angry Men (1957),8.9,12345678910 NOT YET RELEASED Seen,NaN
...,...,...,...,...,...
245,NaN,246. Dersu Uzala (1975),8.0,12345678910 NOT YET RELEASED Seen,NaN
246,NaN,247. Aladdin (1992),8.0,12345678910 NOT YET RELEASED Seen,NaN
247,NaN,248. Gandhi (1982),8.0,12345678910 NOT YET RELEASED Seen,NaN
248,NaN,249. The Help (2011),8.0,12345678910 NOT YET RELEASED Seen,NaN


In [35]:
df = (
    raw_df.assign(
        rank = lambda df: df["Rank & Title"].apply(lambda t: t.split(".", 1)[0]),
        title_and_year = lambda df: df["Rank & Title"].apply(lambda t: t.split(".", 1)[1]),
        year = lambda df: df["title_and_year"].apply(lambda t: t.rsplit("(", 1)[1][:-1]),
        title = lambda df: df["title_and_year"].apply(lambda t: t.rsplit("(", 1)[0].strip()),
    )
    .drop(columns=["Unnamed: 0", "Rank & Title", "Your Rating", "Unnamed: 4", "title_and_year"])
    .rename(columns={"IMDb Rating": "rating"})
)[["rank", "title", "year", "rating"]]
df

,rank,title,year,rating
0,1,The Shawshank Redemption,1994,9.2
1,2,The Godfather,1972,9.2
2,3,The Dark Knight,2008,9.0
3,4,The Godfather Part II,1974,9.0
4,5,12 Angry Men,1957,8.9
...,...,...,...,...
245,246,Dersu Uzala,1975,8.0
246,247,Aladdin,1992,8.0
247,248,Gandhi,1982,8.0
248,249,The Help,2011,8.0


In [36]:
df.to_csv("top_movies.csv", index=False)

## All movie data from IMDB exports

As described in https://www.imdb.com/interfaces/

In [3]:
def download_titles():
    title_basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
    title_basics_raw = pd.read_csv(title_basics_url, sep="\t", low_memory=False, na_values="\\N")
    title_basics = (
        title_basics_raw
        .query("(titleType=='movie') & (isAdult==0)")
        .drop(columns=["isAdult", "endYear", "titleType"])
        .convert_dtypes()
        .astype({"startYear": "Int64", "runtimeMinutes": "Int64"})
    )
    title_basics.to_parquet("imdb_movie_titles.parquet")

if not os.path.isfile("imdb_movie_titles.parquet"):
    download_titles()



ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
ratings_url = ""